In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sanketnadargi","key":"a631beed56e8787f2c504e915b95d6a6"}'}

In [ ]:
!pip install -q kaggle



In [ ]:
!mkdir -p ~/.kaggle


In [ ]:
!cp kaggle.json ~/.kaggle/



In [ ]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mathurinache/the-lj-speech-dataset

100% 2.99G/2.99G [02:32<00:00, 19.5MB/s]
100% 2.99G/2.99G [02:32<00:00, 21.1MB/s]


In [ ]:
!unzip /content/the-lj-speech-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: LJSpeech-1.1/wavs/LJ030-0111.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0112.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0113.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0114.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0115.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0116.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0117.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0118.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0119.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0120.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0121.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0122.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0123.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0124.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0125.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0126.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0127.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0128.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-0129.wav  
  inflating: LJSpeech-1.1/wavs/LJ030-

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display


In [ ]:
df = pd.read_csv('/content/LJSpeech-1.1/metadata.csv',sep='|',header=None)

In [ ]:
df

,0,1,2
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...
...,...,...,...
13095,LJ050-0274,made certain recommendations which it believes...,made certain recommendations which it believes...
13096,LJ050-0275,materially improve upon the procedures in effe...,materially improve upon the procedures in effe...
13097,LJ050-0276,"As has been pointed out, the Commission has no...","As has been pointed out, the Commission has no..."
13098,LJ050-0277,with the active cooperation of the responsible...,with the active cooperation of the responsible...


In [ ]:
df.columns = ['file','trascription','Normalized_trascription']

In [ ]:
df

,file,trascription,Normalized_trascription
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...
...,...,...,...
13095,LJ050-0274,made certain recommendations which it believes...,made certain recommendations which it believes...
13096,LJ050-0275,materially improve upon the procedures in effe...,materially improve upon the procedures in effe...
13097,LJ050-0276,"As has been pointed out, the Commission has no...","As has been pointed out, the Commission has no..."
13098,LJ050-0277,with the active cooperation of the responsible...,with the active cooperation of the responsible...


In [ ]:
df = df[['file','trascription']]
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,file,trascription
0,LJ022-0186,Twenty years of experience with this system ha...
1,LJ013-0243,Good now offered to go to Wandsworth and satis...
2,LJ001-0131,"the size known as ""Long primer"" ought to be th..."
3,LJ036-0049,"And, after we got past Akard, at Murphy -- I f..."
4,LJ039-0188,hit the target twice within a span of 4.6 and ...


In [ ]:
split = int(len(df)*0.90)
df_train = df[:split]
df_val = df[split:]

print(f"Size of training dataset :{len(df_train)}")
print(f"Size of validity dataset :{len(df_val)}")

Size of training dataset :11790
Size of validity dataset :1310


Preprocessing.

In [ ]:
characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]
char_to_num = keras.layers.StringLookup(vocabulary = characters, oov_token="" )
num_to_char = keras.layers.StringLookup(
   vocabulary=char_to_num.get_vocabulary() ,oov_token="",invert = True
)

print(
    f"The vocabulory is : {char_to_num.get_vocabulary()}"
    f"(size={char_to_num.vocabulary_size()})"
)

The vocabulory is : ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' '](size=31)


In [ ]:
pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.6 MB/s eta 0:00:00


In [ ]:
from jiwer import wer

In [ ]:
wavs_path = '/content/LJSpeech-1.1/wavs'

In [ ]:
frame_length = 256
frame_step =160
fft_length =384

def encode_single_sample(wav_file,label):
  file = tf.io.read_file(wavs_path + wav_file + ".wav")
  audio, _ = tf.audio.decode_wav(file)
  audio = tf.squeeze(audio,axis=-1)
  audio = tf.cast(audio, tf.float32)
  spectrogram = tf.signal.stft(
      audio,frame_length =  frame_length , frame_step=frame_step,fft_length=fft_length
   )
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.math.pow(spectrogram,0.5)
  means = tf.math.reduce_mean(spectrogram,1,keepdims=True)
  stddevs= tf.math.reduce_std(spectrogram,1,keepdims=True)
  spectrogram = (spectrogram - means)/(stddevs + 1e-10)
  label = tf.strings.lower(label)
  return spectrogram , label

In [ ]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["file"]),list(df_train["trascription"]))

)
train_dataset = (
    train_dataset.map(encode_single_sample,num_parallel_calls = tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["file"]),list(df_val["trascription"]))
)

validation_dataset = (
    validation_dataset.map(encode_single_sample,num_parallel_calls = tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)


In [ ]:
def CTCLoss(y_true,y_pred):
  batch_len = tf.cast(tf.shape(y_true)[0],dtype="int64")
  input_length = tf.cast(tf.shape(y_pred)[1],dtype="int64")
  label_length = tf.cast(tf.shape(y_true)[1],dtype="int64")

  input_length = input_length * tf.ones(shape=(batch_len,1),dtype="int64")
  label_length = label_length * tf.ones(shape=(batch_len,1),dtype="int64")

  loss = keras.backend.ctc_batch_cost(y_true,y_pred,input_length,label_length)
  return loss

In [ ]:
def build_model(input_dim,output_dim,rnn_layers=5,rnn_units=128):
  input_spectrogram = layers.Input((None,input_dim),name="input")
  x = layers.Reshape((-1,input_dim,1),name="expand_dim")(input_spectrogram)
  # Layer1 Convulution
  x = layers.Conv2D (
      filters=32,
      kernel_size = [11,41],
      strides=[2,2],
      padding="same",
      use_bias=False,
      name="conv1")(x)
  x= layers.BatchNormalization(name="conv1_batch_norm")(x)
  x= layers.ReLU(name="conv1_relu")(x)
    #Layer 2 Convolution

  x = layers.Conv2D(
      filters=32,
      kernel_size = [11,21],
      strides=[1,2],
      padding="same",
      use_bias=False,
      name="conv2",
  )(x)
  x= layers.BatchNormalization(name="conv2_batch_norm")(x)
  x= layers.ReLU(name="conv2_relu")(x)
  #reshaping
  x=layers.Reshape((-1,x.shape[-2]*x.shape[-1]))(x)
  #RNN LAyer
  for i in range(1,rnn_layers+1):
    recurrent =layers.GRU(
        units = rnn_units,
        activation = "tanh",
        recurrent_activation="sigmoid",
        use_bias = True,
        return_sequences = True,
        reset_after = True,
        name = f"gru_{i}",
    )
    x=layers.Bidirectional(
        recurrent,name = f"bidirectional_{i}",merge_mode="concat"
    )(x)
    if i < rnn_layers:
      x=layers.Dropout(rate=0.5)(x)
  #Dense layer
  x = layers.Dense(units=rnn_units *2,name="Dense_Layer")(x)
  x = layers.ReLU(name="dense1_Relu")(x)
  x = layers.Dropout(rate=0.5)(x)

  #Classification Layer
  output = layers.Dense(units=output_dim + 1 , activation="softmax")(x)
  model = keras.Model(input_spectrogram,output,name="DEEPSPEECH")
  opt = keras.optimizers.Adam(learning_rate= 1e-4)
  model.compile(optimizer=opt,loss=CTCLoss)
  return model


model=build_model(
    input_dim=fft_length // 2 + 1,
    output_dim = char_to_num.vocabulary_size(),
    rnn_units=512,
)

model.summary(line_length=110)

Model: "DEEPSPEECH"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 input (InputLayer)                              [(None, None, 193)]                         0                
                                                                                                              
 expand_dim (Reshape)                            (None, None, 193, 1)                        0                
                                                                                                              
 conv1 (Conv2D)                                  (None, None, 97, 32)                        14432            
                                                                                                              
 conv1_batch_norm (BatchNormalization)           (None, None, 97, 32)                       

In [ ]:
def decode_batch_predictions(pred):
      input_len = np.ones(pred.shape[0])* pred.shape[1]
      results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
      output_text=[]
      for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
      return output_text



#A callback class to output a few transcriptions during training
class CallbackEval(keras.callbacks.Callback):
  def __init__(self, dataset):
    super().__init__()
    self.dataset = dataset

  def on_epoch_end(self, epoch: int, logs=None):
    predictions = []
    targets = []
    for batch in self.dataset:
      X, y = batch
      batch_predictions = model.predict(X)
      batch_predictions = decode_batch_predictions(batch_predictions)
      predictions.extend(batch_predictions)
      for label in y:
        label = (
            tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
            )
        targets.append(label)
    wer_score = wer(targets, predictions)
    print("-" * 100)
    print(f"Word Error Rate: {wer_score: 4f}")
    print("-" * 100)
    for i in np.random.randint(0, Len(predictions), 2):
       print(f"Target : {targets[i]}")
       print(f"Prediction:{predictions[i]}")
       print("-"* 100)



In [ ]:
epochs = 2
validation_callback = CallbackEval(validation_dataset)
history=model.fit(
    train_dataset,
    validation_data = validation_dataset,
    epochs = epochs,
    callbacks=[validation_callback],
)

Epoch 1/2


ValueError: ignored